# Library for loading configs
*Pydantic* is the clear winner, because:
- it is flexible (supports multiple file formats, as well as overriding any config value using environment variables);
- it ensures type-safety (and it has many inbuilt validators for things such as URLs or email addresses, so we don't have to reinvent the wheel to check these)

# File format
## JSON 
Advantages: 
- allows **nested** configs, which would be nice to have because that would allow us to:
  - *organize* the configs in a better way, e.g. break up the configs for a single step by where they are needed (e.g., instantiating processor class vs for constructing run_args), which in turn may simplify that code.
  - directly define any nested config *values* right in the config, which again simplifies our code.
- Easier to switch to Terraform template later, which has some other benefits (see below).

## .env file
Advantage: Easier for user to write, compared to JSON.

## Decision
I initially started with .env files because I thought I could work around the limitations of not having nesting. However, it currently looks like having access to nesting would allow for a substantial reduction in code complexity (especially for the step factories), so I'm planning to **switch to JSON**.

# How to break up into multiple files
## By environment?
For now, I'm breaking up the configs into a separate file for each environment. While there are some config values that could be shared across environments, this would introduce too much additional complexity. 

In the medium-term, the goal is to find a way to share config values that we want to be the same across environments, especially those that need to be the same in order to avoid unexpected differences in behavior – e.g. step type (if it does come from a config - ideally we would avoid such potential sources of bugs altogether, such as by making the step type a property of a given processor).

The main way to achieve this goal would be by splitting up the config into a **template** (which can directly set shared values) and use **parameters**  which are populated from a different file (one for each env). Note that the downside of this approach would be that it creates one more file across which configs are scattered, but hopefully this will be offset by the fact that the configs will be more organized.

Todo: Look into ways how to implement this:
- Simple way: Set shared values in a config file, and load an additional .env file that differs by environment
  - Downsides: 
    - Doesn't allow parameter substitution (e.g., change prefix or postfix of a role name or bucket name) - have to set whole value.
    - Doesn't easily allow nested config for anything that differs by environment (since we are bound by the limitations of .env files)
- Use a Terraform template file 
  - Downsides:
    - Have to see how usable that is for local development. (Worst case, we can resort to use pydantic's ability to override all the settings using environment variables for local development; though that too would become tedious if many of the configs are nested).

## By pipeline step
### Separate config for each step
Advantage: Suuport the **Open-Closed principle**: makes the library easier to extend for users, because they can simply add a new file for a new step they want to add, but don't have to modify existing files.

### Single config 
Advantage: slightly easier to load all configs at once, if we wanted to do that (don't have to traverse file paths, though that wouldn't be a big deal either).
Downside: **users would have to modify existing config definition in order to extend the library, which severely limits the reusability of this library**, because it is impossible to foresee all necessary use cases (and even if he could, it would be infeasible to implement all these centrally).

### Decision
Clearly, we have to use a **separate config file for each step**.